 # Import package 

In [8]:
import sys
import numpy as np
import pandas as pd
import csv

# Read in training set

In [31]:
raw_data = np.genfromtxt('train.csv', delimiter=',', encoding='utf8') ## train.csv
#raw_data = np.loadtxt('train.csv', delimiter=',')
data = raw_data[1:,3:]
where_are_NaNs = np.isnan(data)
data[where_are_NaNs] = 0 
month_to_data = {}  ## Dictionary (key:month , value:data)                                  

# generate month_to_data(20 days data)
for month in range(12):
    sample = np.empty(shape = (18 , 480))
    for day in range(20):
        for hour in range(24): 
            sample[:,day * 24 + hour] = data[18 * (month * 20 + day): 18 * (month * 20 + day + 1),hour]
    month_to_data[month] = sample  

# Preprocess

In [32]:
x = np.empty(shape = (12 * 471 , 18 * 9),dtype = float)
y = np.empty(shape = (12 * 471 , 1),dtype = float)

for month in range(12): 
    for day in range(20): 
        for hour in range(24):   
            if day == 19 and hour > 14:
                continue  
            x[month * 471 + day * 24 + hour,:] = month_to_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1) 
            y[month * 471 + day * 24 + hour,0] = month_to_data[month][9 ,day * 24 + hour + 9]
print(x.shape)
print(y.shape)

(5652, 162)
(5652, 1)


# Normalization

In [33]:
mean = np.mean(x, axis = 0) 
std = np.std(x, axis = 0)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        if not std[j] == 0 :
            x[i][j] = (x[i][j]- mean[j]) / std[j]

# Training

In [34]:
dim = x.shape[1] + 1 
w = np.zeros(shape = (dim, 1 ))
x = np.concatenate((np.ones((x.shape[0], 1 )), x) , axis = 1).astype(float)
learning_rate = np.array([[200]] * dim)
adagrad_sum = np.zeros(shape = (dim, 1 ))
 
for T in range(10000):
    if(T % 500 == 0 ):
        print("T=",T)
        print("Loss:",np.power(np.sum(np.power(x.dot(w) - y, 2 ))/ x.shape[0],0.5))
    gradient = (-2) * np.transpose(x).dot(y-x.dot(w))
    adagrad_sum += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad_sum) + 0.0005)

np.save('weight.npy',w)     ## save weight

T= 0
Loss: 27.071214829194115
T= 500
Loss: 15.303358672239382
T= 1000
Loss: 9.95733625358027
T= 1500
Loss: 7.879329888549471
T= 2000
Loss: 6.861485712171391
T= 2500
Loss: 6.339003480940456
T= 3000
Loss: 6.060560050269416
T= 3500
Loss: 5.906939151605037
T= 4000
Loss: 5.819427069726109
T= 4500
Loss: 5.768117525889321
T= 5000
Loss: 5.737251324218732
T= 5500
Loss: 5.718247893619341
T= 6000
Loss: 5.70629462196756
T= 6500
Loss: 5.698621259127973
T= 7000
Loss: 5.693596789769688
T= 7500
Loss: 5.690241610223954
T= 8000
Loss: 5.687956629045993
T= 8500
Loss: 5.686369228004014
T= 9000
Loss: 5.685243884510883
T= 9500
Loss: 5.684429390954338


In [ ]:
# Read in testing set

In [36]:
w = np.load('weight.npy')                                   ## load weight
test_raw_data = np.genfromtxt('test.csv', delimiter=',')   ## test.csv
test_data = test_raw_data[:, 2: ]
where_are_NaNs = np.isnan(test_data)
test_data[where_are_NaNs] = 0 

In [ ]:
# Predict

In [39]:
test_x = np.empty(shape = (240, 18 * 9),dtype = float)

for i in range(240):
    test_x[i,:] = test_data[18 * i : 18 * (i+1),:].reshape(1,-1) 

for i in range(test_x.shape[0]):        ##Normalization
    for j in range(test_x.shape[1]):
        if not std[j] == 0 :
            test_x[i][j] = (test_x[i][j]- mean[j]) / std[j]

test_x = np.concatenate((np.ones(shape = (test_x.shape[0],1)),test_x),axis = 1).astype(float)
answer = test_x.dot(w)

In [40]:
# 存储文件 Write file

In [47]:
f = open('result.csv',"w")
w = csv.writer(f)
title = ['id','value']
w.writerow(title) 
for i in range(240):
    content = ['id_'+str(i),answer[i][0]]
    w.writerow(content) 